In [ ]:
# установим нужную библиотеку
!pip install tinvest[cli]

# 1. Получение токена и проверка соединения с сервером

Ссылка на github, откуда берется библиотека: https://github.com/daxartio/tinvest

Ссылка на документацию по использованию API Tinkoff: https://tinkoffcreditsystems.github.io/invest-openapi/

Устанавливаем необходимую библиотеку. Далее нужно получить токен. Как это сделать - описано здесь: https://tinkoffcreditsystems.github.io/invest-openapi/auth/

Я записал токен в отдельный файлик и подгружаю его, чтобы не светить в коде

In [2]:
import tinvest
import pandas as pd
import numpy as np
from decimal import Decimal
from tinkoff_token import token # токен хранится здесь

In [3]:
TOKEN = token
client = tinvest.SyncClient(TOKEN)

response = client.get_portfolio()  # tinvest.PortfolioResponse
#print(response.payload)

Напишем примитивную функцию проверки обработки нашего запроса

In [4]:
def check_response_status(response):
    if response.status == 'Ok':
        return 'Запрос обработан'
    return 'Ошибка обработки запроса'

In [5]:
check_response_status(response)

'Запрос обработан'

Токен работает, сервер отвечает.

Теперь получим `id` брокерского аккаунта. У меня их два: ИИС и обычный. Всё самое интересное лежит на обычном аккаунте, поэтому возьмем его и по этому `id` обновим ответ сервера

In [6]:
broker_account_id='2038650350'
response = client.get_portfolio(broker_account_id='2038650350')

In [7]:
#response

Видим, что в `response` лежит не человекочитаемый формат. Распарсим его, приведя в итоге к классическому `pandas.DataFrame`

In [8]:
all_positions = response.payload.positions

# 2. Получение, преобразование и анализ данных по портфелю

Для этого напишем простую функцию под каждый интересующий нас параметр, вызовем и результат работы поместим в DataFrame.

Список интересующих нас параметров:

<li> Position - наименование финансового инструмента 
<li> Instrument_type - тип финансового инструмента (stock, bond, etf, currency)
<li> Balance - количество бумаг в портфеле
<li> Avg_position_price - усредненная цена покупки
<li> Total_price - цена актива. Рассчитывается как balance * avg_position_price
<li> Profit - величина дохода в случае продажи в абсолютных единицах (может быть и отрицательной)
<li> Profit_percent - величина дохода в случае продажи в процентах (может быть и отрицательной)
<li> Currency - валюта
<li> Ticker - тикер финансового инструмента

P.S. Profit обновляется каждый раз при новом запросе к серверу

In [9]:
def get_all_position_names(all_positions):
    names = []
    for position in all_positions:
        names.append(position.name)
    return names

In [10]:
def get_instrument_type(all_positions):
    instrument_type = []
    for position in all_positions:
        instrument_type.append(position.instrument_type.replace('InstrumentType', ''))
    return instrument_type

In [11]:
def get_balance(all_positions):
    balance = []
    for position in all_positions:
        balance.append(position.balance)
    return balance

In [12]:
def get_avg_position_price(all_positions):
    avg_position_price = []
    for position in all_positions:
        avg_position_price.append(position.average_position_price.value)
    return avg_position_price

In [13]:
def get_current_price(avg_position_price, profit_percent):
    current_price = []
    for i, elem in enumerate(avg_position_price):
        current_price.append(avg_position_price[i] + (avg_position_price[i] * (profit_percent[i] / 100)))
    return current_price

In [14]:
def get_total_price(balance, current_price):
    total_price = []
    for i, elem in enumerate(balance):
        total_price.append((balance[i] * current_price[i]).quantize(Decimal('1.00')))
    return total_price

In [15]:
def get_expected_profit(all_positions):
    profit = []
    for positions in all_positions:
        profit.append(positions.expected_yield.value)
    return profit

In [16]:
def get_expected_profit_percent(profit, total_price):
    profit_percent = []
    for i, elem in enumerate(profit):
        #profit_percent.append(f'{(profit[i] / total_price[i] * 100).quantize(Decimal("1.00"))}%')
        profit_percent.append((profit[i] / total_price[i] * 100).quantize(Decimal("1.00")))
    return profit_percent

In [17]:
def get_currency(all_positions):
    currency = []
    for position in all_positions:
        currency.append(position.average_position_price.currency.replace('Currency.', ''))
    return currency

In [18]:
def get_ticker(all_positions):
    tickers = []
    for position in all_positions:
        tickers.append(position.ticker)
    return tickers

In [19]:
def update_expected_profit(all_positions):
    return get_expected_profit(all_positions)

Мы написали все необходимые для начала функции. Следующим шагом их нужно вызвать. Запишем результат работы каждой функции в соответсвующую переменную и затем передадим в датафрейм

In [30]:
name = get_all_position_names(all_positions)
instrument_type = get_instrument_type(all_positions)
balance = get_balance(all_positions)
avg_position_price = get_avg_position_price(all_positions)
currency = get_currency(all_positions)
tickers = get_ticker(all_positions)
#profit = get_expected_profit(all_positions) # расчет прибыли в первый раз
profit = update_expected_profit(all_positions) # обновление прибыли
total_price = get_total_price(balance, avg_position_price) # временный костыль для размыкание последующих трех строк
profit_percent = get_expected_profit_percent(profit, total_price)
current_price = get_current_price(avg_position_price, profit_percent)
total_price = get_total_price(balance, current_price)

In [31]:
portfolio = pd.DataFrame({'position': name, 'instrument_type': instrument_type,
                    'balance': balance, 'avg_position_price': avg_position_price,
                    'current_price': current_price,
                    'total_price': total_price, 'profit': profit, 
                     'profit_percent': profit_percent,
                     'currency': currency, 'ticker': tickers
                    })

In [32]:
portfolio

,position,instrument_type,balance,avg_position_price,current_price,total_price,profit,profit_percent,currency,ticker
0,Полюс Золото,Stock,1,17156,14182.8652,14182.87,-2972.5,-17.33,RUB,PLZL
1,Polymetal,Stock,3,1962.4,1622.70856,4868.13,-1019.1,-17.31,RUB,POLY
2,ГДР X5 RetailGroup,Stock,3,2808.5,2475.97360,7427.92,-997.5,-11.84,RUB,FIVE
3,Аэрофлот,Stock,20.0,73.9,68.8009,1376.02,-102,-6.90,RUB,AFLT
4,Alibaba,Stock,1,263.3,205.97959,205.98,-57.33,-21.77,USD,BABA
5,Уралкалий выпуск 6,Bond,6,1031.77,982.657748,5895.95,-294.6,-4.76,RUB,RU000A101GZ6
6,Тинькофф Банк выпуск 3,Bond,5,1052.86,1032.85566,5164.28,-100.0,-1.90,RUB,RU000A100V79
7,О'КЕЙ выпуск 3,Bond,5,1018.72,996.817520,4984.09,-109.5,-2.15,RUB,RU000A1014B9
8,Гарант-Инвест БО 001P-05,Bond,1,762.59,794.847557,794.85,32.23,4.23,RUB,RU000A1005T9
9,FinEx Акции компаний IT-сектора США,Etf,8,1.0975,1.26750275,10.14,1.36,15.49,USD,FXIM


Данные получены и преобразованы в более понятный для человека формат, однако не хватает множества вспомогательных величин. Например:
<li> Общий объём портфеля (в рублях и долларах)
<li> Профит портфеоя в абсолютных единицах (в рублях и долларах)
<li> Остаток валюты (в рубли и долларах) </li>
    


In [23]:
def get_portfolio_currency():
    currencies = client.get_portfolio_currencies() # получаем список валют: eur, rub, usd
    rub = currencies.payload.currencies[1].balance # выбираем rub
    usd = currencies.payload.currencies[2].balance # выбираем usd
    return rub, usd

In [24]:
def get_course_usd_to_rub():
    return (client.get_market_orderbook(figi='BBG0013HGFT4', depth=1)).payload.asks[0].price # получаем текущий курс доллара

In [25]:
def get_total_portfolio_rub(portfolio):
    rub, usd = get_portfolio_currency() # получаем остаток рублей и долларов в портфеле
    course_usd_to_rub = get_course_usd_to_rub() # получаем актуальный курс доллара
    rub_position = portfolio.loc[portfolio['currency'] == 'RUB']['total_price'].sum() # складываем все рублевые активы
    usd_position = portfolio.loc[portfolio['currency'] == 'USD']['total_price'].sum() # складываем все долларовые активы
    total_sum = rub + rub_position + course_usd_to_rub * (usd + usd_position) # суммируем все и переводим доллары в рубли
    return f'Объём портфеля в рублях: {total_sum.quantize(Decimal("1.00"))}р'

In [27]:
get_total_portfolio_rub(portfolio)

'Объём портфеля в рублях: 75408.86р'

In [ ]:
rub, usd = get_portfolio_currency() # получаем остаток рублей и долларов в портфеле
course_usd_to_rub = get_course_usd_to_rub() # получаем актуальный курс доллара
rub_position = portfolio.loc[portfolio['currency'] == 'RUB']['total_price'].sum()
usd_position = portfolio.loc[portfolio['currency'] == 'USD']['total_price'].sum()
total_sum = rub + rub_position + (course_usd_to_rub * usd_position)
total_sum

In [ ]:
rub_position = portfolio.loc[portfolio['currency'] == 'RUB']['total_price'].sum()

In [ ]:
rub_position

In [ ]:
usd_position

In [ ]:
def get_total_portfolio_usd():
    u
    return usd, rub, eur

In [ ]:
kek = client.get_portfolio_currencies().payload.currencies #eur rub usd

In [ ]:
course_usd = (client.get_market_orderbook(figi='BBG0013HGFT4', depth=1)).payload.asks[0].price
#usd = course_usd.payload.asks[0].price

In [ ]:
course_usd

In [ ]:
client.get_market_search_by_ticker('PLZL')

In [ ]:
# TODO
# 1. Operation list
# 2. Smth with taxes

In [ ]:
client??

In [ ]:
tinvest.MarketOrderRequest??